In [ ]:
zoom chat text from maggie for sql



select prop.parcelid, prop.fips, txn.transactiondate, pred.logerror
from properties_2017 prop

join (
select parcelid, max(transactiondate) as transactiondate
from predictions_2017
group by parcelid
) txn using(parcelid)

join predictions_2017 pred using(parcelid, transactiondate)
;





Use zillow;



select parcelid, count(*)
from predictions_2017
group by parcelid
having count(*) > 1;

select count(*) from predictions_2017;

select count(*) from
(
select parcelid, max(transactiondate) as transactiondate
from predictions_2017
group by parcelid
) a;